# Setup AND import

In [1]:
from pathlib import Path
import sys

# Find project root (folder that contains `src`)
ROOT = Path.cwd()
while not (ROOT / "src").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

SRC = str(ROOT / "src")
if SRC not in sys.path:
    sys.path.insert(0, SRC)

print("PROJECT ROOT:", ROOT)
print("SRC PATH:", SRC)





PROJECT ROOT: c:\Users\Mona\OneDrive\Desktop\week2-data-work
SRC PATH: c:\Users\Mona\OneDrive\Desktop\week2-data-work\src


In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

from bootcamp_data.config import make_paths

paths = make_paths()
DATA = paths.processed
FIGS = paths.figures.resolve()

print("DATA:", DATA)
print("FIGS:", FIGS)


DATA: C:\Users\Mona\OneDrive\Desktop\week2-data-work\data\processed
FIGS: C:\Users\Mona\OneDrive\Desktop\week2-data-work\reports\figures


In [4]:
from pathlib import Path

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires `kaleido`)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)


# loading the file 
### And printing a small summery about the data 

In [27]:
df = pd.read_parquet(DATA / "analytics_table.parquet")
print("Number Of Rows : " , " " , len(df))
print("-----------------------------------------------------------")
print("Data Types : " , "\n" , df.dtypes.head(15))
print("-----------------------------------------------------------")

print("Top missing values : " , " " , df.isna().sum().sort_values(ascending=False).head())
print("-------------------------------------------------------------------")
 
df.describe()

Number Of Rows :    15
-----------------------------------------------------------
Data Types :  
 order_id               string[python]
user_id                string[python]
amount                        Float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
data                           object
year                          float64
month                  string[python]
day                            object
hour                          float64
country                string[python]
dtype: object
-----------------------------------------------------------
Top missing values :    quantity             3
amount               3
amount_is_outlier    3
amount_winsor        3
hour                 2
dtype: int64
-------------------------------------------------------------------


,amount,quantity,year,hour,signup_date,amount_winsor
count,12.0,12.0,13.0,13.000000,15,12.0
mean,62.1875,1.583333,2025.0,11.846154,2025-11-24 20:48:00,61.312083
min,8.0,1.0,2025.0,8.000000,2025-11-15 00:00:00,8.495
25%,22.6875,1.0,2025.0,10.000000,2025-11-23 12:00:00,22.6875
50%,57.5,1.5,2025.0,11.000000,2025-11-26 00:00:00,57.5
75%,82.5,2.0,2025.0,14.000000,2025-11-28 00:00:00,82.5
max,200.0,3.0,2025.0,17.000000,2025-11-30 00:00:00,189.0
std,53.691789,0.668558,0.0,2.794225,NaN,51.112549


# Qustions 
- How does the total order amount change over time (daily)? 
- What is the average order amount per country?
- How many orders are refunded or have missing data, and how does this vary by country?

# Qustion 1 :  How does the total order amount change over time (daily)? 


# Insight from total order amount change over time
-  From December 1st to 6th, revenue increased gradually from 20.5 to 50 SAR, indicating healthy early momentum.
- A dramatic spike to 189.0  occurred on December 7th—likely driven by a large bulk order, a promotional event.
- Revenue plummeted to 0 SAR on December 8th, immediately after the high. This could reflect a data recording gap.
### Caveat : 
- One possible caveat is that the data may be sparse or incomplete for some days (like Dec 2 and Dec 8), which could affect the accuracy of the trend. Always double-check for missing or erroneous entries. 

In [10]:
#First we need to create a new timestamp day becouse we will groupby day
df["Day"]=df["created_at"].dt.date
#print(df["Day"])
daily_revenu=(df.groupby("Day" , dropna=False).agg(num_orders=("order_id","size"), revenu=("amount_winsor" , "sum"))
                                              .reset_index().sort_values("Day"))
print(daily_revenu.head(10))

#Now we will create the plots 
#I will be using A line beacouse it shows trends overtime 
fig1 = px.line(
    daily_revenu,
    x="Day",
    y="revenu",

    title="Total Order Amount Over Time (Daily)"
)

fig1
save_fig(fig1, FIGS / "Total_amo_change_daily.png")




          Day  num_orders  revenu
0  2025-12-01           2  20.995
1  2025-12-02           1     0.0
2  2025-12-03           1    25.0
3  2025-12-05           1    50.0
4  2025-12-06           1   15.75
5  2025-12-07           1   189.0
6  2025-12-08           1     0.0
7  2025-12-09           1    30.0
8  2025-12-10           1    70.0
9  2025-12-11           1    65.0


# Qustion 2 : What is the average order amount per country?


# Average Order Amount by Country – What I Found
- UK had the highest spend per order – around 135 SAR on average. That means either people there are buying pricier items, or they place fewer orders but spend more each time.
- Saudi Arabia (SA) had the most orders (7) but a lower average spend of ~42 SAR. this mean customers there buy more often, but spend less per order.
- UAE and US had the lowest average order values, at ~40 SAR and ~48 SAR respectively. They also weren’t the lowest in total orders, so it’s not a lack of traffic—it’s about how much people spend each time.

### Caveat:
- Countries like Egypt and UK have very few orders, which can make their averages misleading or not representative

In [ ]:
#Creating the table
#First we need to groupby contry 
avg_amo_co=(df.groupby("country" , dropna=False).agg(order_num=("order_id" , "size") , avg_amount=("amount" , "mean")).reset_index())
print(avg_amo_co.head(10))

#Creating the graph
fig2 = px.bar(avg_amo_co , x="country" , y="avg_amount"  , title="Average Order Amount per Country")


fig2
save_fig(fig2, FIGS / "average_order_by_country.png")


# Qustion 3 : How many orders are refunded or have missing data, and how does this vary by country?

# How many orders are refunded or have missing data, and how does this vary by country? - My finddings
 - Some countries show high rates of problematic orders, especially AE and UK.

 - There are refunded orders and missing values, which affect overall data quality,These issues can impact the reliability of downstream analysis and should be addressed

 - SA has a moderate level of problematic orders.

 - US and EG show zero problem rates, but this may be due to small sample sizes.


In [24]:

is_refunded = df["status_clean"].eq("refund")
has_missing = df[["amount__isna", "quantity__isna"]].any(axis=1)

# A copy becauese to be safe :)
df_q = df.copy()
df_q["problem_order"] = is_refunded | has_missing


by_country = (
    df_q.groupby("country", dropna=False)
        .agg(
            total_orders=("order_id", "size"),
            problem_orders=("problem_order", "sum"),
        )
        .assign(problem_rate=lambda x: x["problem_orders"] / x["total_orders"])
        .reset_index()
)

print(by_country)

#creating the fig 
fig = px.imshow(
    by_country.set_index("country")[["problem_rate"]],
    text_auto=".2f",
    color_continuous_scale="Reds",
    title="Problem Order Rate (Refunded or Missing) by Country"
)

fig


save_fig(fig, FIGS / "problem_order_rate_heatmap.png")


  country  total_orders  problem_orders  problem_rate
0      AE             3               2      0.666667
1      EG             1               0           0.0
2      SA             7               3      0.428571
3      UK             2               1           0.5
4      US             2               0           0.0


# Bootstrap

In [ ]:
db = df.copy()

db["is_refund"] = db["status_clean"].eq("refund").astype(int)


a = db.loc[db["country"] == "SA", "is_refund"]
b = db.loc[db["country"] == "AE", "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))

# n_boot -----> number of time we want to preffurm the bootstap
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)

    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()

    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())

    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }

res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
print(res)


n_SA: 7 n_AE: 3
{'diff_mean': -0.5238095238095237, 'ci_low': -1.0, 'ci_high': 0.14285714285714285}
